In [ ]:
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import f1_score,accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import warnings
warnings.filterwarnings("ignore")

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
series_ids = pd.read_parquet('./Dataset/Zzzs_train.parquet', columns=['series_id'])
series_ids = series_ids.series_id.unique()

In [ ]:
def data_preprocessing(df, window):
    rog_args = {'window' : window, 'min_periods':2}
    df[f'anglez_bf_{window}_std'] = df.anglez.rolling(**rog_args).std().round(4)
    df[f'anglez_at_{window}_std'] = df.anglez.iloc[::-1].rolling(**rog_args).std().round(4).sort_index()
    df[f'anglez_bf_{window}_mean'] = df.anglez.rolling(**rog_args).mean().round(4)
    df[f'anglez_at_{window}_mean'] = df.anglez.iloc[::-1].rolling(**rog_args).mean().round(4).sort_index()
    df[f'enmo_bf_{window}_std'] = df.enmo.rolling(**rog_args).std().round(4)
    df[f'enmo_at_{window}_std'] = df.enmo.iloc[::-1].rolling(**rog_args).std().round(4).sort_index()
    df[f'enmo_bf_{window}_mean'] = df.enmo.rolling(**rog_args).mean().round(4)
    df[f'enmo_at_{window}_mean'] = df.enmo.iloc[::-1].rolling(**rog_args).mean().round(4).sort_index()
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df

In [ ]:
train_series_ids = series_ids
train_list = []
window_list = [60, 360, 720] # 5m, 30m, 1h
for _id in tqdm(train_series_ids):
    train_df_tmp = pd.read_parquet("./Dataset/Zzzs_train.parquet", filters=[('series_id','=',_id)], columns = ['anglez', 'enmo', 'awake'])
    train_df_tmp.anglez = (train_df_tmp.anglez + 8.8104) / 35.5218
    train_df_tmp.enmo = (train_df_tmp.enmo - 0.0413) / 0.1018
    for window in window_list:
        train_df_tmp = data_preprocessing(train_df_tmp, window)
    train_list.append(train_df_tmp)

train = pd.concat(train_list, ignore_index=True)
train_x = train.drop('awake',axis = 1)
train_y = train[['awake']]
del train, train_list

In [ ]:
# 원본 DataFrame의 열 이름 저장
column_names = train_x.columns.tolist()

# 시퀀스 길이를 정의
sequence_length = 24  # 시퀀스 길이를 24로 설정

# train_x의 첫 번째 차원이 sequence_length로 나누어 떨어지도록 train_x를 잘라냄
train_x = train_x[:-(train_x.shape[0] % sequence_length)]

# 데이터 형태 변경
train_x = np.array(train_x).reshape(-1, sequence_length, train_x.shape[1])
train_y = np.array(train_y)

# 모델 정의
model = Sequential()

# LSTM 레이어 추가
model.add(LSTM(128, input_shape=(sequence_length, train_x.shape[2]), activation='tanh', recurrent_activation='sigmoid', recurrent_dropout=0, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='tanh', recurrent_activation='sigmoid', recurrent_dropout=0))
model.add(Dropout(0.2))


# 출력 레이어 추가
model.add(Dense(1, activation='sigmoid'))  # awake 필드가 이진 분류라고 가정

# 모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_x, train_y, epochs=3, validation_split=0.2)

In [ ]:
def get_events(_id, model, file_root = None):
    test = pd.read_parquet(f"{file_root}", filters=[('series_id','=',_id)])
    test['timestamp'] = pd.to_datetime(test['timestamp']).apply(lambda t: t.tz_localize(None))
    test['date'] = test['timestamp'].dt.date
    test['hour'] = test['timestamp'].dt.hour
    test.anglez = (test.anglez + 8.8104) / 35.5218
    test.enmo = (test.enmo - 0.0413) / 0.1018
    for window in window_list:
        test = data_preprocessing(test,window)
    test.dropna(inplace=True)
    test.reset_index(drop=True,inplace=True)
    test_x = test[column_names]
    
    # 데이터 형태 변경
    test_x = np.array(test_x).reshape(-1, sequence_length, test_x.shape[1])

    # RNN 모델을 사용한 예측
    preds_probs = model.predict(test_x)
    preds = np.round(preds_probs).flatten()  # 이진 분류 결과를 얻기 위해 반올림
    probs = preds_probs.flatten()

    # preds와 probs를 별도의 데이터프레임으로 생성
    preds_df = pd.DataFrame(preds, columns=['prediction'])
    probs_df = pd.DataFrame(probs, columns=['probability'])

    # preds와 probs 데이터프레임을 test 데이터프레임과 병합
    test = pd.concat([test, preds_df, probs_df], axis=1)
    
    test = test[test['prediction']!=2]
    test.loc[test['prediction']==0, 'probability'] = 1-test.loc[test['prediction']==0, 'probability']
    test['score'] = test['probability'].rolling(60*12*5, center=True, min_periods=10).mean().bfill().ffill()
    test['pred_diff'] = test['prediction'].diff()
    test['event'] = test['pred_diff'].replace({1:'wakeup', -1:'onset', 0:np.nan})
    test.loc[test.hour.isin([0,1,2,3,4,5,6]),'date'] = test.loc[test.hour.isin([0,1,2,3,4,5,6]),'date'] + pd.Timedelta(days=-1)
    test_wakeup = test[test['event']=='wakeup'].groupby(test['timestamp'].dt.date).agg('first')
    test_onset = test[test['event']=='onset'].groupby(test['date']).agg('last')
    test = pd.concat([test_wakeup, test_onset], ignore_index=True).sort_values('timestamp')
    return test

In [ ]:
# 예측 및 제출 파일 생성
file_root = './Dataset/test_series.parquet'
series_id  = pd.read_parquet(file_root, columns=['series_id'])
series_id = series_id.series_id.unique()
submit_columns = ['series_id','step','event','score']
submission = []

for _id in series_id:
    test_tmp = get_events(_id, model, file_root)
    test_tmp = test_tmp[submit_columns]
    submission.append(test_tmp)

submission = pd.concat(submission, ignore_index=True).reset_index(names='row_id')
submission.to_csv('submission.csv', index=False)

In [ ]:
submission